In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense

In [4]:
import psycopg2
import pandas as pd

# Database connection details
username = 'tradingview'
password = 'tradingview123'
hostname = '192.168.1.142'
port = '5432'
database_name = 'pindb'

# Establish connection
conn = psycopg2.connect(
    dbname=database_name,
    user=username,
    password=password,
    host=hostname,
    port=port
)

# SQL query
sql_query = "SELECT  TO_TIMESTAMP(lasttradetime) AT TIME ZONE 'Asia/Kolkata' AS time, open,high,low,close,openinterest FROM whole_indices_futures WHERE instrumentidentifier = 'NIFTY16MAY22050CE' order by lasttradetime;"

# Execute SQL query
cursor = conn.cursor()
cursor.execute(sql_query)

# Fetch all the results
rows = cursor.fetchall()

# Convert to DataFrame
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Close cursor and connection
cursor.close()
conn.close()

# Display the DataFrame
print(df)


OperationalError: connection to server at "192.168.1.142", port 5432 failed: FATAL:  no pg_hba.conf entry for host "192.168.1.30", user "tradingview", database "pindb", SSL encryption
connection to server at "192.168.1.142", port 5432 failed: FATAL:  no pg_hba.conf entry for host "192.168.1.30", user "tradingview", database "pindb", no encryption


In [1]:
df.head()

NameError: name 'df' is not defined

: 

In [ ]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
lookback_window =3
features = 11
def create_sequences(data, lookback_window):  

    X, y = [], []
    for i in range(len(data) - lookback_window - 1): 
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, 0:11]) 
    return np.array(X), np.array(y)

model = Sequential()
model.add(LSTM(units=200, return_sequences=True, input_shape=(lookback_window, features)))
model.add(LSTM(units=200))
model.add(Dense(200, activation=tf.nn.leaky_relu))
model.add(Dense(200, activation=tf.nn.leaky_relu))
model.add(Dense(units=11)) 
model.compile(loss='mean_squared_error', optimizer='adam')
X, y = create_sequences(df_scaled, lookback_window)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]




model.fit(X_train, y_train, epochs=50, batch_size=30, validation_data=(X_test, y_test))
y_pred = model.predict(X_test)
y_pred_original = scaler.inverse_transform(y_pred)
data = pd.DataFrame(y_pred_original, columns=['PrevClose','OpenPrice','HighPrice','LowPrice','LastPrice','ClosePrice','AveragePrice','TotalTradedQuantity','Turnover','DeliverableQty','trade'])
data 


In [ ]:
# Define the function to generate predictions
def generate_predictions(model, last_sequence, num_predictions):
    predictions = []
    current_sequence = last_sequence.reshape(1, lookback_window, features)  # Reshape to match model input shape
    for _ in range(num_predictions):
        next_prediction = model.predict(current_sequence)  # Predict the next data point
        predictions.append(next_prediction[0])  # Append the prediction to the list of predictions
        current_sequence = np.append(current_sequence[:, 1:, :], next_prediction.reshape(1, 1, features), axis=1)  # Update the current sequence
    return predictions

# Generate predictions for the next 720 data points
last_sequence = X_test[-1]  # Using the last sequence from the test data
num_predictions = 375
predicted_data = generate_predictions(model, last_sequence, num_predictions)

# Inverse transform the predictions to obtain the original scale
predicted_data_original = scaler.inverse_transform(predicted_data)

# Create a DataFrame with the predicted values and column names
predicted_df = pd.DataFrame(predicted_data_original, columns=['PrevClose','OpenPrice','HighPrice','LowPrice','LastPrice','ClosePrice','AveragePrice','TotalTradedQuantity','Turnover','DeliverableQty','trade'])

# Assuming the last timestamp in the existing data is '2024-04-09 15:30:00', and you want to generate predictions for the next 720 minutes (12 hours)
last_timestamp = pd.Timestamp('2024-04-09 15:30:00')
predicted_df['time'] = pd.date_range(sta=lrtast_timestamp + pd.Timedelta(minutes=1), periods=num_predictions, freq='T')

# Print the predicted data
print(predicted_df)
print(predicted_df['close'].max(),predicted_df['close'].min())